In [4]:
#userid = '127682630593203'
disk_path = 'D:/Group1'
baseurl = 'https://graph.facebook.com/v2.4/'
access_token = 'CAACEdEose0cBAEXmo0KO4F4WxBwmhXxooQ0nCLGujfPV2jcOAslRL1Y5qjnB9OKyESuZAolhFquFB2tb7gMzkpBV3gNmdI0N2cSDn7YcRdGYnsbyAabjPGCkrUauFGahXC6FST73SoMnQ5mTIZBvGkFYiJCXUL15NDJpwLFLibGHwjbjUMuD8oWM072K7i3DTFqFsEhzZALele9TZAT6'

In [6]:
import requests
import json
import os
import re

#創專案資料夾
def create():
    if not os.path.exists(disk_path):
        os.makedirs(disk_path)
    if not os.path.exists(disk_path + '/pic'):
        os.makedirs(disk_path + '/pic')
    if not os.path.exists(disk_path + '/comments'):
        os.makedirs(disk_path + '/comments')
    if not os.path.exists(disk_path + '/message'):
        os.makedirs(disk_path + '/message')

#抓取粉絲團名單作為userid
def get_userid(url,access_token):
    url_graph = url.replace('www','graph') + '?access_token=' + access_token    
    res = requests.get(url_graph)
    data = res.json()
    return data['id']

#抓取粉絲團ID
def fb_ids(url,access_token):
    header = {'Authorization':'Bearer {}'.format(access_token)}
    res = requests.get(url,headers = header)
    data = res.json()
    return data['id']

#抓取json(一開始查詢的連接)
def url_res(url,access_token):
    header = {'Authorization':'Bearer {}'.format(access_token)}
    res = requests.get(url,headers = header)
    data = res.json()
    return data['posts']

#讀取json(next的連接)
def url_res_noposts(url):
    res = requests.get(url)
    data = res.json()
    return data

#不知道為甚麼nexturl抓出來只有一半,所以就加上access_token
def next_url(url,access_token):
    return url + '&access_token=' + access_token

#將picture抓取下來,依照頁數分類,並按粉絲團ID存放
def img_parser(data,fbid): 
    if not os.path.exists(disk_path + '/pic/' + fbid):
        os.makedirs(disk_path + '/pic/' + fbid)
    for data in data['data']:
        if 'full_picture' in data:
            pic_url = data['full_picture']
            filename = data['id']
            m = re.search('(?P<picname>[\d_]+)',filename)
            if m:
                picname = m.group('picname')
            pic = open(disk_path + '/pic/' + fbid + '/{}.jpg'.format(picname),'wb')
            pic.writelines(requests.get(pic_url).content)
            pic.close()

#將message抓取
def message(data,fbid):
    if not os.path.exists(disk_path + '/message/' + fbid):
        os.makedirs(disk_path + '/message/' + fbid)
    for data in data['data']:
        if 'message' in data:
            message = data['message']
            filename = data['id']
            m = re.search('(?P<messname>[\d_]+)',filename)
            if m:
                messname = m.group('messname')
            mess = open(disk_path + '/message/' + fbid + '/{}.txt'.format(messname),'w')
            mess.write(message.encode('utf8'))
            mess.close()

#抓取評論
def comments(data,fbid):
    if not os.path.exists(disk_path + '/comments/' + fbid):
        os.makedirs(disk_path + '/comments/' + fbid)
    for ele in data['data']:
        commlist = []
        if 'comments' in ele:
            filename = ele['id']
            for data in ele['comments']['data']:
                message = data['message']
                m = re.search('(?P<commname>[\d_]+)',filename)
                commlist.append(data['created_time'].encode('utf8')+'\n'+data['from']['id'].encode('utf8')+'\n'+data['message'].encode('utf8')+'\n'+'================================================='+'\n')
                if m:
                    commname = m.group('commname')
            comm = open(disk_path + '/comments/' + fbid + '/{}.txt'.format(commname),'w')
            for ele in commlist:
                comm.write(ele)
            comm.close()

#判斷是否還有下一頁
def nextfrompage(data):
    data = data['paging']
    next_page=data['next']
    if next_page is None:
        return None
    else:
        return True

#抓取下一頁連接
def nexturl(data):
    data = data['paging']
    next_page=data['next']
    return next_page

#主程式
def main():
    create()
    #取得粉絲團連結
    f = open(disk_path + '/fanpages.txt','r')
    while(True):
        fanpage_link = f.readline()
        if fanpage_link == '':
            break
        else:
            userid = get_userid(fanpage_link,access_token)
    
            #設定連結
            res_url = baseurl + userid + '?fields=id,posts{full_picture,message,message_tags,likes{name},created_time}'
            comm_url = baseurl + userid + '?fields=posts{comments}'
    
            data = url_res(res_url,access_token)
            fb_id = fb_ids(res_url,access_token)
            comm_data = url_res(comm_url,access_token)
            next_page = nextfrompage(data)

            while(next_page):
                img_parser(data,fb_id)
                message(data,fb_id)
                comments(comm_data,fb_id)
                #判斷下一頁及抓取下一頁連接
                next_page = nextfrompage(data)
                normal_next_url = nexturl(data)
                comm_next_url = nexturl(comm_data)
                #取得下一頁的json
                data = url_res_noposts(next_url(normal_next_url,access_token))
                comm_data = url_res_noposts(next_url(comm_next_url,access_token))
    f.close()

if __name__ == '__main__':
    main()

KeyboardInterrupt: 